In [ ]:
import numpy as np
import random
import math
import numdifftools
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def F(z): 
    rx = abs(z[0] - center[0])
    ry = abs(z[1] - center[1])# Random point in the cluster
    alpha = 1.5 # Consistent with the density (As this gets larger, less samples are moved close to 0)
    xval = rx * math.erf(rx/alpha) + (alpha/math.sqrt(pi)) * math.pow(e, -(rx/alpha) ** 2)
    yval = ry * math.erf(ry/alpha) + (alpha/math.sqrt(pi)) * math.pow(e, -(ry/alpha) ** 2)
    return (xval, yval)

def F_1(z): 
    r = abs(z - RandomSample)# Random point in the cluster
    alpha = 1.5 # Consistent with the density (As this gets larger, less samples are moved close to 0)
    return r * math.erf(r/alpha) + (alpha/math.sqrt(pi)) * math.pow(e, -(r/alpha) ** 2)

def BetaCalculation():
    Proportion = 0.5
    xSummationDerivative = [0, 0]
    ySummationDerivative = [0, 0]
    Beta = []
    for j in range(0, len(StandardNormal)):
        #CurTuple = F(StandardNormal[j])
        ySummationDerivative[0] += F(StandardNormal[j])[0]
        ySummationDerivative[1] += F(StandardNormal[j])[1]
    for i in range(0, len(MixtureSample)):
        xSummationDerivative[0] += F(MixtureSample[i])[0]
        xSummationDerivative[1] += F(MixtureSample[i])[1]
    Beta.append((-1/len(MixtureSample)) * xSummationDerivative[0] + \
        (1/len(StandardNormal)) * ySummationDerivative[0])
    Beta.append((-1/len(MixtureSample)) * xSummationDerivative[1] + \
        (1/len(StandardNormal)) * ySummationDerivative[1])
    return np.multiply(Beta, Proportion)

def u(x, Beta):
    xval = (x[0] ** 2 / 2) + Beta[0] * F(x)[0] + Beta[0] * F(x)[0]
    yval = (x[1] ** 2 / 2) + Beta[1] * F(x)[1] + Beta[0] * F(x)[1]
    return (xval, yval)

def uConjugate(y, Beta):
    ConvexCandidate = []
    for i in range(0, len(MixtureSamples)):
        xval = (MixtureSamples[i][0] * y) - u(MixtureSamples[i], Beta)[0]
        yval = (MixtureSamples[i][1] * y) - u(MixtureSamples[i], Beta)[1]
        ConvexCandidate.append((xval, yval))
    SumList = []
    for i in range(len(ConvexCandidate)):
        SumList = ConvexCadidate[i][0] + ConvexCandidate[i][1]
    index = SumList.index(max(SumList))
    return ConvexCandidate[index]

def SamplesUpdate(OldMixtureSample):
    NewMixtureSample = []
    for i in range(0, len(OldMixtureSample)):
        xval = OldMixtureSample[i][0] + Beta[0] * numdifftools.Gradient(F_1)([OldMixtureSample[i][0]]) + Beta[0] * numdifftools.Gradient(F_1)([OldMixtureSample[i][0]])
        yval = OldMixtureSample[i][1] + Beta[1] * numdifftools.Gradient(F_1)([OldMixtureSample[i][1]]) + Beta[0] * numdifftools.Gradient(F_1)([OldMixtureSample[i][1]])
        NewMixtureSample.append([xval, yval])
    NewMixtureSample = np.array(NewMixtureSample)
    return NewMixtureSample

def MixtureSampleGenerator():
    mean1 = [1, -1]
    cov1 = [[1, 0], [0, 1]]
    mean2 = [-1, 1]
    cov2 = [[1, 0], [0, 1]]
    x = np.random.multivariate_normal(mean1, cov1, 500)
    y = np.random.multivariate_normal(mean2, cov2, 500)
    MixtureSample = []
    for i in range(500):
        RandomSelector = random.random()
        if RandomSelector > 0.7:
            MixtureSample.append(x[i])
        else:
            MixtureSample.append(y[i])
    MixtureSample = np.array(MixtureSample)
    return MixtureSample

def StandardNormalGenerator():
    Sample = []
    x = np.random.standard_normal(500)
    y = np.random.standard_normal(500)
    for i in range(500):
        Sample.append([x[i], y[i]])
    return Sample

MixtureSample = MixtureSampleGenerator()
plt.scatter(*zip(*MixtureSample), color = 'b', alpha = 0.5)
StandardNormal = StandardNormalGenerator()

for i in range(50): # Maybe there is a problem of overfitting
    center = StandardNormal[random.randint(0, len(StandardNormal) - 1)]
    Beta = BetaCalculation()
    MixtureSample = SamplesUpdate(MixtureSample)

plt.scatter(*zip(*MixtureSample), color = 'r', alpha = 0.5)
plt.xlim(-6, 6)
plt.ylim(-4, 4)
plt.show()
